# Brighway activities

__Difficulty:__ Basic :)

In [51]:
import bw2data
from bw2data.backends import Activity
from bw2data.backends.proxies import Exchanges

In [52]:
# Change this to your local setup
BW_EXAMPLE_PROJECT = "ecoinvent_391"

In [53]:
bw2data.projects.set_current(BW_EXAMPLE_PROJECT)
# which databases does the project have
bw2data.databases

Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent_391_cutoff

In [54]:
# Change this to the ecoinvent or other database that has some technosphere activities
BW_EXAMPLE_DB = "ecoinvent_391_cutoff"

In [55]:
from bw2data.backends import SQLiteBackend

db: SQLiteBackend = bw2data.Database(BW_EXAMPLE_DB)
db, f"Activities: {len(db)}"

(Brightway2 SQLiteBackend: ecoinvent_391_cutoff, 'Activities: 21238')

In [56]:
activity: Activity = db.random()
activity, type(activity)

('market for zeolite, slurry, without water, in 50% solution state' (kilogram, RoW, None),
 bw2data.backends.proxies.Activity)

In [28]:
def print_exchanges(exchange_list: Exchanges):
    for exchange in exchange_list:
        print(exchange)
        print()
    print()

In [30]:
# biosphere inputs
biosphere_exchanges: Exchanges = activity.biosphere()
print(f"*** biosphere exchanges ({len(biosphere_exchanges)})")
print_exchanges(biosphere_exchanges)

# technosphere inputs
technosphere_exchanges = activity.technosphere()
print(f"*** technosphere exchanges ({len(technosphere_exchanges)})")
print_exchanges(technosphere_exchanges)

# technosphere outputs
upstream_exchanges: Exchanges = activity.exchanges()
# exactly the same as: activity.consumers()
print(f"*** upstream exchanges ({len(upstream_exchanges)})")
print_exchanges(upstream_exchanges)

# product
production_exchange: Exchanges = activity.production()
# exactly the same as: activity.producers
print(f"*** production exchange (generally (always) 1)")
print(list(production_exchange)[0])

*** biosphere exchanges (0)

*** technosphere exchanges (6)
Exchange: 0.769230769230769 kilogram 'Sohio process' (kilogram, RoW, None) to 'market for acetonitrile' (kilogram, GLO, None)>

Exchange: 0.230769230769231 kilogram 'Sohio process' (kilogram, RER, None) to 'market for acetonitrile' (kilogram, GLO, None)>

Exchange: 0.3091 ton kilometer 'market group for transport, freight train' (ton kilometer, GLO, None) to 'market for acetonitrile' (kilogram, GLO, None)>

Exchange: 0.0246 ton kilometer 'market group for transport, freight, inland waterways, barge' (ton kilometer, GLO, None) to 'market for acetonitrile' (kilogram, GLO, None)>

Exchange: 0.2088 ton kilometer 'market group for transport, freight, lorry, unspecified' (ton kilometer, GLO, None) to 'market for acetonitrile' (kilogram, GLO, None)>

Exchange: 0.599 ton kilometer 'market for transport, freight, sea, container ship' (ton kilometer, GLO, None) to 'market for acetonitrile' (kilogram, GLO, None)>


*** upstream exchanges

In [31]:
# all inputs at once this includes the biosphere, technosphere and the production
all_inputs: Exchanges = activity.exchanges()
# exactly the same as activity.edges()
print(f"*** all inputs ({len(list(all_inputs))})")
for exchange in upstream_exchanges:
    print(exchange)
    print()

*** all inputs (7)
Exchange: 1.0 kilogram 'market for acetonitrile' (kilogram, GLO, None) to 'market for acetonitrile' (kilogram, GLO, None)>

Exchange: 0.769230769230769 kilogram 'Sohio process' (kilogram, RoW, None) to 'market for acetonitrile' (kilogram, GLO, None)>

Exchange: 0.230769230769231 kilogram 'Sohio process' (kilogram, RER, None) to 'market for acetonitrile' (kilogram, GLO, None)>

Exchange: 0.3091 ton kilometer 'market group for transport, freight train' (ton kilometer, GLO, None) to 'market for acetonitrile' (kilogram, GLO, None)>

Exchange: 0.0246 ton kilometer 'market group for transport, freight, inland waterways, barge' (ton kilometer, GLO, None) to 'market for acetonitrile' (kilogram, GLO, None)>

Exchange: 0.2088 ton kilometer 'market group for transport, freight, lorry, unspecified' (ton kilometer, GLO, None) to 'market for acetonitrile' (kilogram, GLO, None)>

Exchange: 0.599 ton kilometer 'market for transport, freight, sea, container ship' (ton kilometer, GLO,

In [34]:
first_input = next(iter(all_inputs))
print(first_input.input)
print(first_input.output)

'market for acetonitrile' (kilogram, GLO, None)
'market for acetonitrile' (kilogram, GLO, None)


In [35]:
second_input = list(all_inputs)[1]
print(second_input.input)
print(second_input.output)

'Sohio process' (kilogram, RoW, None)
'market for acetonitrile' (kilogram, GLO, None)


In [62]:

def collect_background_activities(activity: Activity, levels: int = 1) -> list[set[Activity]]:
    """
    
    :param activity: 
    :param levels: pass -1 for search until no new activities are added
    :return: 
    """
    background_system: list[set[Activity]] = [{activity}]
    all_activities: set[Activity] = {activity}
    current_level = 0
    # use a while loop, because passing levels: -1 means it goes on until there is nothing new to add
    while True:
        print(f"level: {current_level}")
        current_level_activities = background_system[current_level]
        next_level_activities: set[Activity] = set()
        for _activity in current_level_activities:
            technosphere_input_exchanges: Exchanges = _activity.exchanges()
            for input_exchange in technosphere_input_exchanges:
                if input_exchange.input not in all_activities:
                    next_level_activities.add(input_exchange.input)
        print(f"Adding {len(next_level_activities)} activities")
        background_system.append(next_level_activities)
        if len(next_level_activities) > 0:
            all_activities.update(next_level_activities)
        # end the loop, if no new activities have been added
        if not next_level_activities:
            break
        if levels != -1 and current_level == levels:
            break
        current_level += 1
    return background_system


collect_background_activities(activity, -1)

level: 0
Adding 12 activities
level: 1
Adding 2207 activities
level: 2
Adding 247 activities
level: 3
Adding 709 activities
level: 4
Adding 1317 activities
level: 5
Adding 2880 activities
level: 6
Adding 2755 activities
level: 7
Adding 2357 activities
level: 8
Adding 1135 activities
level: 9
Adding 1079 activities
level: 10
Adding 610 activities
level: 11
Adding 514 activities
level: 12
Adding 268 activities
level: 13
Adding 181 activities
level: 14
Adding 75 activities
level: 15
Adding 52 activities
level: 16
Adding 35 activities
level: 17
Adding 55 activities
level: 18
Adding 140 activities
level: 19
Adding 49 activities
level: 20
Adding 70 activities
level: 21
Adding 33 activities
level: 22
Adding 40 activities
level: 23
Adding 15 activities
level: 24
Adding 21 activities
level: 25
Adding 6 activities
level: 26
Adding 6 activities
level: 27
Adding 0 activities


[{'market for zeolite, slurry, without water, in 50% solution state' (kilogram, RoW, None)},
 {'market for transport, freight, light commercial vehicle' (ton kilometer, RoW, None),
  'market for transport, freight, lorry, unspecified' (ton kilometer, RoW, None),
  'market for transport, freight train' (ton kilometer, ZA, None),
  'market for transport, freight train' (ton kilometer, IN, None),
  'zeolite production, slurry, without water, in 50% solution state' (kilogram, RoW, None),
  'market for transport, freight, lorry, unspecified' (ton kilometer, ZA, None),
  'market for transport, freight train' (ton kilometer, US, None),
  'market for transport, freight train' (ton kilometer, CN, None),
  'market for transport, freight, sea, container ship' (ton kilometer, GLO, None),
  'market for transport, freight train' (ton kilometer, RoW, None),
  'market for transport, freight, lorry, unspecified' (ton kilometer, BR, None),
  'market for transport, freight, light commercial vehicle' (ton